In [2]:
import pandas as pd
import requests
import json
from pathlib import Path

In [54]:
# Load existing data
old_eip_projects = pd.read_excel("../../../data/raw/2023.05.24 OGW database.xlsx", sheet_name = "Project")
transformed_data = pd.read_parquet('../../../data/output/data_warehouse/eip_facilities.parquet')

In [8]:
fac_proj = pd.read_parquet('../../../data/output/data_warehouse/eip_facility_project_association.parquet')

,air_construction_id,name,raw_created_on,raw_modified_on,raw_date_last_checked,raw_project_id,raw_permit_status,description_or_purpose,raw_application_date,raw_draft_permit_issuance_date,raw_last_day_to_comment,raw_final_permit_issuance_date,raw_deadline_to_begin_construction,detailed_permitting_history,document_url,date_modified,permit_status
0,1573,7482,2021-05-20T19:18:48.607217,2021-11-30T14:22:59.283119,<NA>,2723,Final Issued,Initial construction permit for the 3 Bear Lib...,2017-09-11,<NA>,<NA>,2018-01-08,<NA>,<NA>,<NA>,2021-11-30 14:22:59.283,Final Issued
1,1574,7482M1,2021-05-20T19:18:48.607217,2021-11-23T16:32:26.201611,<NA>,2723,Final Issued,"Permit modification to add a generator, correc...",2019-09-11,<NA>,<NA>,2020-04-08,<NA>,<NA>,<NA>,2021-11-23 16:32:26.201,Final Issued
2,1575,3120-00129-00,2021-05-20T19:18:48.607217,2022-01-07T16:01:12.876381,<NA>,2728,Final Issued,Permit authorizing initial construction of the...,2020-03-09,<NA>,<NA>,2020-05-21,<NA>,3120-00129-00 (issued 5/21/2020),<NA>,2022-01-07 16:01:12.876,Final Issued
3,1578,73110,2021-05-20T19:18:48.607217,2023-02-01T21:51:11.178899,<NA>,2727,Final Issued,Permit authorizing initial construction of Ste...,2020-07-10,<NA>,<NA>,2021-02-05,<NA>,<NA>,<NA>,2023-02-01 21:51:11.178,Final Issued
4,1581,AQ1524CPT01,2021-05-20T19:18:48.607217,2022-03-03T19:26:56.597437,2022-03-03,2875,Final Issued,This permit authorized construction of a gas t...,2017-12-28,<NA>,<NA>,2020-08-13,2023-08-13,AQ1524CPT01 (issued 8/13/2020) - applied for a...,<NA>,2022-03-03 19:26:56.597,Final Issued
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,6321,139561 - Second Extension,2023-04-25T20:26:48.156254,2023-04-25T20:37:38.544775,2023-04-25,3121,Application Pending,Application for a second 18-month extension to...,2023-04-20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2023-04-25 20:37:38.544,Application Pending
941,6322,PN-22-012,2023-04-26T20:51:51.884989,2023-04-26T21:01:23.345727,2023-04-26,2861,Draft Issued,This permit authorizes as-built changes and mo...,2022-01-10,2023-03-09,2023-04-08,<NA>,<NA>,<NA>,<NA>,2023-04-26 21:01:23.345,Draft Issued
942,6345,57352,2023-04-28T19:41:49.383406,2023-05-01T19:34:45.575250,2023-04-28,6344,Application Pending,This permit would authorize construction of th...,2023-04-18,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2023-05-01 19:34:45.575,Application Pending
943,6386,3120-00059-V4,2023-05-15T17:45:40.923975,2023-05-15T18:00:09.554546,2023-05-15,6385,Final Issued,This permit authorizes construction of a secon...,2021-11-04,<NA>,<NA>,2022-04-21,<NA>,<NA>,<NA>,2023-05-15 18:00:09.554,Final Issued


## Get new data from API

The data is served through a xata API underlying the website. We query this directly as it provides access to the normalized data, and the new Excel file doesn't contain all the information we need.

In [4]:
eip_data_dictionary = {
    'facilities': '01-00_FACILITIES',
    'projects': '01-03_PROJECTS',
    'facility_project_assn': '01-03-0_link_Projects_Facilities',
    'air_construction_permits': '03-00_AIR_CONSTRUCTION',
    'air_construction_project_assn': '03-00-0_link_AirConstruction_Projects'
}

In [ ]:
# Set path for downloading new data for check
data_dir = Path().resolve()

PosixPath('/home/thinky/Desktop/Catalyst/PUDL/deployment-gap-model/notebooks/data_updates/eip_infrastructure')

In [6]:
def get_fossil_infrastructure_data(entity: str):
    dfs = []
    more = True
    body = {"page":{"size":999}}
    
    entity_endpoint = eip_data_dictionary[entity]
    while more is True: # While there are more pages to download
        response = requests.post(f"https://oilandgaswatch.org/xata-api/{entity_endpoint}/query", json = body)
        response_json = response.json()
        breakpoint()
        df = pd.DataFrame(response_json['records'])
        dfs.append(df)
        # Check for more records
        more = response_json['meta']['page']['more']
        body['page']['after'] = response_json['meta']['page']['cursor']

    df_combined = pd.concat(dfs).reset_index(drop = True)
    df_combined.to_csv(f"eip_{entity}_{pd.to_datetime('today').date()}.csv", index = False)

In [7]:
for key in eip_data_dictionary:
    get_fossil_infrastructure_data(entity = key)

## Test data processing

Process the new data.

In [6]:
from dbcp.extract.eip_infrastructure import extract

raw_dfs = extract('/app/data/raw/eip_infrastructure')

/app/.local/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/app/.local/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/app/.local/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from

In [4]:
from dbcp.transform.eip_infrastructure import facilities_transform

fac_df = facilities_transform(raw_dfs['eip_facilities'])

In [4]:
from dbcp.transform.eip_infrastructure import projects_transform

proj_df = projects_transform(raw_dfs['eip_projects'])

In [8]:
proj_df.columns

Index(['raw_actual_operating_year', 'raw_actual_or_expected_completion_year',
       'carbon_monoxide_co_tpy', 'raw_classification',
       'coal_plant_co2e_equivalency', 'gas_powered_vehicles_equivalency',
       'greenhouse_gases_co2e_tpy', 'hazardous_air_pollutants_haps_tpy',
       'hazardous_air_pollutants_haps_potential_lbspy', 'raw_industry_sector',
       'mortality_cost_of_additional_co2e_deaths', 'nitrogen_oxides_nox_tpy',
       'raw_operating_status', 'operating_status_source_documents',
       'operating_status_source_documents_old', 'particulate_matter_pm2_5_tpy',
       'raw_permitting_and_emissions_accounting_notes', 'project_description',
       'raw_project_type', 'published', 'research_notes',
       'social_cost_of_additional_co2e_lower_estimate',
       'social_cost_of_additional_co2e_upper_estimate',
       'sulfur_dioxide_so2_tpy', 'volatile_organic_compounds_voc_tpy',
       'created_by', 'project_id', 'unknown_id', 'modified_by', 'project_name',
       'raw_pro

In [5]:
from dbcp.transform.eip_infrastructure import air_construction_transform

air_df = air_construction_transform(raw_dfs['eip_air_construction_permits'])

In [7]:
from dbcp.transform.eip_infrastructure import facilities_project_assn_transform, _split_json_column

fac_proj_df = facilities_project_assn_transform(raw_dfs['eip_facility_project_assn'])

In [5]:
from dbcp.transform.eip_infrastructure import project_permit_assn_transform

proj_perm = project_permit_assn_transform(raw_dfs['eip_air_construction_project_assn'])

In [9]:
[col for col in fac_proj_df.columns if col not in fac_proj.columns]

['connection_id',
 'connection_unknown_id',
 'xata_created_at',
 'xata_updated_at',
 'xata_version']

In [18]:
proj_perm[['connection_id',
 'connection_unknown_id',
 'xata_created_at',
 'xata_updated_at',
 'xata_version']]

,connection_id,connection_unknown_id,xata_created_at,xata_updated_at,xata_version
0,rec_cn5s4050jhd1hc94ue3g,11485,2024-02-13T19:24:48.976373Z,2024-04-08T18:03:37.440425Z,3
1,rec_cn5s407rusrcll7ljho0,11486,2024-02-13T19:24:48.777114Z,2024-04-08T18:03:37.564329Z,3
2,rec_cn5s40d0jhd1hc94ue40,11487,2024-02-13T19:24:49.523451Z,2024-04-08T18:03:37.668962Z,3
3,rec_cn5s40frusrcll7ljhog,11488,2024-02-13T19:24:49.158772Z,2024-04-08T18:03:37.79488Z,3
4,rec_cn5s40frusrcll7ljhp0,11489,2024-02-13T19:24:49.348046Z,2024-04-08T18:03:37.89403Z,3
...,...,...,...,...,...
1008,rec_ct7ii9m4gbv0p11jq3vg,15150,2024-12-03T15:52:06.138998Z,2024-12-03T17:47:46.265715Z,1
1009,rec_ctbk8flqfkdtj0lqmveg,15173,2024-12-09T19:25:50.326739Z,2024-12-10T03:28:02.796726Z,1
1010,rec_ctcrl6rifh7okq995rf0,15190,2024-12-11T16:15:23.462361Z,2024-12-11T17:46:23.804994Z,1
1011,rec_ctit1hrifh7okq9nc24g,15234,2024-12-20T20:17:11.666155Z,2024-12-21T03:02:59.893008Z,1


In [12]:
proj_df[proj_df.project_id == "5417"].published

706    True
Name: published, dtype: bool

In [43]:
import pandas as pd
col = 'xata'
json_cols = pd.json_normalize(df['xata'].map(eval))
json_cols = json_cols.add_prefix('xata_')
json_cols
df = pd.concat([df, json_cols], axis="columns")
df.drop(col, axis="columns")

col = 'Facility'
prefix = "facility_"
json_cols = pd.json_normalize(df[col].map(eval))

In [46]:
json_cols = json_cols.add_prefix(prefix)
df = pd.concat([df, json_cols], axis="columns")

In [48]:
df.drop(col, axis="columns")

,Project,id,id_qaqc,xata,xata_createdAt,xata_updatedAt,xata_version,xata_createdAt,xata_updatedAt,xata_version,...,xata_createdAt,xata_updatedAt,xata_version,xata_createdAt,xata_updatedAt,xata_version,xata_createdAt,xata_updatedAt,xata_version,facility_facility_facility_facility_id
0,{'id': '3008'},rec_cn5ruml0jhd1hc94u8q0,10296,"{'createdAt': '2024-02-13T19:13:30.555196Z', '...",2024-02-13T19:13:30.555196Z,2024-04-08T18:01:08.560465Z,3,2024-02-13T19:13:30.555196Z,2024-04-08T18:01:08.560465Z,3,...,2024-02-13T19:13:30.555196Z,2024-04-08T18:01:08.560465Z,3,2024-02-13T19:13:30.555196Z,2024-04-08T18:01:08.560465Z,3,2024-02-13T19:13:30.555196Z,2024-04-08T18:01:08.560465Z,3,1001
1,{'id': '3007'},rec_cn5rumnrusrcll7livmg,10297,"{'createdAt': '2024-02-13T19:13:30.287755Z', '...",2024-02-13T19:13:30.287755Z,2024-04-08T18:01:08.658793Z,3,2024-02-13T19:13:30.287755Z,2024-04-08T18:01:08.658793Z,3,...,2024-02-13T19:13:30.287755Z,2024-04-08T18:01:08.658793Z,3,2024-02-13T19:13:30.287755Z,2024-04-08T18:01:08.658793Z,3,2024-02-13T19:13:30.287755Z,2024-04-08T18:01:08.658793Z,3,1000
2,{'id': '3009'},rec_cn5rumnrusrcll7livn0,10298,"{'createdAt': '2024-02-13T19:13:30.766339Z', '...",2024-02-13T19:13:30.766339Z,2024-04-08T18:01:08.818406Z,3,2024-02-13T19:13:30.766339Z,2024-04-08T18:01:08.818406Z,3,...,2024-02-13T19:13:30.766339Z,2024-04-08T18:01:08.818406Z,3,2024-02-13T19:13:30.766339Z,2024-04-08T18:01:08.818406Z,3,2024-02-13T19:13:30.766339Z,2024-04-08T18:01:08.818406Z,3,1002
3,{'id': '3012'},rec_cn5rumt0jhd1hc94u8qg,10299,"{'createdAt': '2024-02-13T19:13:31.194114Z', '...",2024-02-13T19:13:31.194114Z,2024-04-08T18:01:08.943714Z,3,2024-02-13T19:13:31.194114Z,2024-04-08T18:01:08.943714Z,3,...,2024-02-13T19:13:31.194114Z,2024-04-08T18:01:08.943714Z,3,2024-02-13T19:13:31.194114Z,2024-04-08T18:01:08.943714Z,3,2024-02-13T19:13:31.194114Z,2024-04-08T18:01:08.943714Z,3,1005
4,{'id': '3017'},rec_cn5rumt0jhd1hc94u8r0,10300,"{'createdAt': '2024-02-13T19:13:31.728011Z', '...",2024-02-13T19:13:31.728011Z,2024-04-08T18:01:09.06002Z,3,2024-02-13T19:13:31.728011Z,2024-04-08T18:01:09.06002Z,3,...,2024-02-13T19:13:31.728011Z,2024-04-08T18:01:09.06002Z,3,2024-02-13T19:13:31.728011Z,2024-04-08T18:01:09.06002Z,3,2024-02-13T19:13:31.728011Z,2024-04-08T18:01:09.06002Z,3,1008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,{'id': 'rec_ctld29qab52bjranujjg'},rec_ctld3raab52bjranun80,15254,"{'createdAt': '2024-12-24T15:23:25.21814Z', 'u...",2024-12-24T15:23:25.21814Z,2024-12-24T17:43:21.271233Z,1,2024-12-24T15:23:25.21814Z,2024-12-24T17:43:21.271233Z,1,...,2024-12-24T15:23:25.21814Z,2024-12-24T17:43:21.271233Z,1,2024-12-24T15:23:25.21814Z,2024-12-24T17:43:21.271233Z,1,2024-12-24T15:23:25.21814Z,2024-12-24T17:43:21.271233Z,1,rec_ctkshhf86psc0rgul2o0
1241,{'id': 'rec_ctrgg3v86psc0rh9flcg'},rec_ctrggciab52bjrb14e4g,15264,"{'createdAt': '2025-01-02T21:42:10.704544Z', '...",2025-01-02T21:42:10.704544Z,2025-01-03T03:05:18.282914Z,1,2025-01-02T21:42:10.704544Z,2025-01-03T03:05:18.282914Z,1,...,2025-01-02T21:42:10.704544Z,2025-01-03T03:05:18.282914Z,1,2025-01-02T21:42:10.704544Z,2025-01-03T03:05:18.282914Z,1,2025-01-02T21:42:10.704544Z,2025-01-03T03:05:18.282914Z,1,rec_ctrfiln86psc0rh9c1qg
1242,{'id': 'rec_cts1pqn86psc0rhb62og'},rec_cts1q3rifh7okqa90lrg,15270,"{'createdAt': '2025-01-03T17:23:27.450975Z', '...",2025-01-03T17:23:27.450975Z,2025-01-03T17:44:14.712287Z,1,2025-01-03T17:23:27.450975Z,2025-01-03T17:44:14.712287Z,1,...,2025-01-03T17:23:27.450975Z,2025-01-03T17:44:14.712287Z,1,2025-01-03T17:23:27.450975Z,2025-01-03T17:44:14.712287Z,1,2025-01-03T17:23:27.450975Z,2025-01-03T17:44:14.712287Z,1,rec_cts1p0bifh7okqa90f5g
1243,{'id': 'rec_cts2pr2ab52bjrb2p6l0'},rec_cts2q5786psc0rhbaimg,15282,"{'createdAt': '2025-01-03T18:31:48.272067Z', '...",2025-01-03T18:31:48.272067Z,2025-01-04T03:03:26.347561Z,1,2025-01-03T18:31:48.272067Z,2025-01-04T03:03:26.347561Z,1,...,2025-01-03T18:31:48.272067Z,2025-01-04T03:03:26.347561Z,1,2025-01-03T18:31:

In [26]:
proj_df.unknown_id

0        2723
1        2724
2        2725
3        2726
4        2727
        ...  
1239    15253
1240    15263
1241    15269
1242    15281
1243    15295
Name: unknown_id, Length: 1244, dtype: Int32

In [13]:
new_eip_air.columns

Index(['State', 'County or Parish', 'Company', 'Facility',
       'Facility Description', 'Project', 'Operating Status',
       'Status Last Checked', 'Project Description', 'Project Classification',
       'Industry Sector', 'Project Type', 'Product Type',
       'CO2e PTE (tons/year)', 'PM2.5 PTE (tons/year)', 'NOx PTE (tons/year)',
       'VOC PTE (tons/year)', 'SO2 PTE (tons/year)', 'CO PTE (tons/year)',
       'HAPs PTE (lbs/year)', 'Latitude', 'Longitude', 'EJScreen Report Link',
       'Estimated population - 3 miles', 'Percent people of color - 3 miles',
       'Percent low income - 3 miles', 'ECHO Link 1', 'ECHO Link 2',
       'Company OGW ID', 'Facility OGW ID', 'Project OGW ID',
       'Facility OGW URL'],
      dtype='object')

In [5]:
import pandas as pd
transformed_data = pd.read_parquet('../../../data/output/data_warehouse/eip_air_constr_permits.parquet')

In [6]:
transformed_data.columns

Index(['air_construction_id', 'name', 'raw_created_on', 'raw_modified_on',
       'raw_date_last_checked', 'raw_project_id', 'raw_permit_status',
       'description_or_purpose', 'raw_application_date',
       'raw_draft_permit_issuance_date', 'raw_last_day_to_comment',
       'raw_final_permit_issuance_date', 'raw_deadline_to_begin_construction',
       'detailed_permitting_history', 'document_url', 'date_modified',
       'permit_status'],
      dtype='object')

In [6]:
old_eip

,id,name,created_on,modified_on,CCS/CCUS,CCS (ID),CCS,Company (ID),Company,Project (ID),...,Percent People over 64 Years Old within 3 miles,Air Toxics Cancer Risk (NATA Cancer Risk),Respiratory Hazard Index,PM2.5 (ug/m3),O3 (ppb),Wastewater Discharge Indicator,Location,Facility Footprint,EPA FRS ID,Facility ID
0,739,3 Bear Libby Gas Plant,2021-05-20T19:13:45.411472,2023-02-06T20:40:53.636126,True,5100.0,3 Bear Libby Gas Plant - CCS Project[5100],2447,"3 Bear Delaware Operating – NM, LLC[2447]",2723,...,15,20,0.20,7.32,55.1,2.400000e-11,POINT(-103.525728 32.542358),"POLYGON((-103.528453 32.540066,-103.528423 32....","110070159076, 110070789268",10000.0
1,740,3 Brothers Compressor Station,2021-05-20T19:13:45.411472,2023-02-06T20:41:43.779072,NaN,NaN,NaN,2592,"MarkWest Liberty Midstream & Resources, LLC[2592]",2724,...,25,38,0.30,8.44,44.2,3.000000e-02,POINT(-80.380335 40.331198),POINT(-80.380335 40.331198),110055589616,10221.0
2,741,50 Buttes Gas Plant,2021-05-20T19:13:45.411472,2023-02-06T20:41:54.913539,NaN,NaN,NaN,2697,"Thunder Creek Gas Services, LLC[2697]",2725,...,7,20,0.20,3.93,47.8,1.100000e-03,POINT(-105.77927 43.85124),NaN,110056978169,10358.0
3,742,Agrium US Borger Nitrogen Operations,2021-05-20T19:13:45.411472,2023-02-07T02:43:55.022835,NaN,NaN,NaN,2624,Agrium US Inc[2624],2726,...,14,20,0.21,6.51,52.0,2.300000e-03,POINT(-101.422777 35.641666),POINT(-101.422777 35.641666),110056955576,10266.0
4,743,Albany Compressor Station,2021-05-20T19:13:45.411472,2023-02-06T20:42:16.411585,NaN,NaN,NaN,2659,"Sabal Trail Transmission, LLC[2659]",2729,...,14,40,0.54,8.32,34.8,8.200000e-08,POINT(-84.250549 31.541712),NaN,110070119840,10311.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,6394,Lone Star CCS Hub,2023-05-22T14:36:02.875077,2023-05-23T16:02:19.584259,True,5036.0,BP & Linde CCS[5036],"6395, 5362","BP plc[6395], Linde Inc[5362]",6393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
762,6396,Magnolia Sequestration Hub,2023-05-22T15:22:56.065726,2023-05-22T15:41:01.064250,True,NaN,NaN,"6397, 2626","Magnolia Sequestration Hub LLC[6397], Occident...",6398,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
763,6400,Longleaf CCS Hub,2023-05-22T16:06:10.257152,2023-05-23T16:05:58.503996,True,NaN,NaN,6401,Tenaska[6401],6399,...,22,30,0.50,8.87,36.4,6.000000e-03,POINT(-88.01492 30.999682),NaN,NaN,NaN
764,6402,SkyNRG Washington Plant,2023-05-22T17:24:01.492154,2023-05-22T17:47:34.647224,NaN,NaN,NaN,6403,SkyNRG[6403],6404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
new_eip

,State,Company,Pipeline,Pipeline Description,Operating Status,Status Last Checked,Project Type,Product Type,Miles,Additional Capacity,Product Units,Associated Facilities,Associated Pipelines,Company OGW ID,Pipeline OGW ID,Associated Facilities OGW ID,Associated Pipelines OGW ID,Pipeline OGW URL
0,LA,Driftwood Pipeline LLC,Driftwood LNG Pipeline,The proposed Driftwood Pipeline Project would ...,Proposed,2022-02-16T00:00:00Z,New Pipeline,Natural Gas,99.4,3954.0,MMscf/d,"Basile Compressor Station, Woodside Louisiana ...","Driftwood Line 200 and Line 300 Project, Permi...",2506,3209,"755, 845, 880, 953","4199, 3348",https://oilandgaswatch.org/pipeline/3209
1,"IL, IN","Natural Gas Pipeline Company of America, LLC",134th Street Lateral Project,The 134th Street Lateral Project included a 1....,Operating,2022-06-16T00:00:00Z,Lateral,Natural Gas,1.4,70.0,MMscf/d,NaN,NaN,2613,3257,NaN,NaN,https://oilandgaswatch.org/pipeline/3257
2,MA,"Tennessee Gas Pipeline Company, LLC",261 Upgrade Projects,The 261 Upgrade Project delivers 72.4 million ...,Operating,2022-04-08T00:00:00Z,Expansion,Natural Gas,2.1,72.4,MMscf/d,Compressor Station 261,NaN,2689,3258,814,NaN,https://oilandgaswatch.org/pipeline/3258
3,LA,"Columbia Gulf Transmission, LLC, Kinder Morgan...",Acadiana Pipeline Project,The Acadiana Project increased firm north-to-s...,Operating,2022-04-14T00:00:00Z,Expansion,Natural Gas,0.0,894.0,MMscf/d,"Sabine Pass LNG Terminal, Eunice Compressor St...","Louisiana XPress Project, Driftwood Line 200 a...","2487, 3385",3259,"1047, 861","3319, 4199",https://oilandgaswatch.org/pipeline/3259
4,PA,Adelphia Gateway LLC,Adelphia Gateway Project,The Adelphia Gateway project converted the rem...,Operating,2023-09-14T00:00:00Z,Conversion,Natural Gas,4.7,250.0,MMscf/d,"Marcus Hook Compressor Station, Quakertown Com...",NaN,3386,3260,"3815, 3833",NaN,https://oilandgaswatch.org/pipeline/3260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,LA,Shell Oil Product US,Rome Pipeline,The Rome Pipeline is a proposed offshore pipel...,Announced,2024-11-21T17:18:50.045Z,New Pipeline,Oil,100.0,NaN,NaN,NaN,NaN,6180,rec_csvmmujt683r8q9cc030,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_csvmmu...
310,"AL, GA, MS",Southern Natural Gas Company LLC,South System Expansion 4 (SSE4) Project,The South System Expansion 4 Project (SSE4) wo...,Proposed,2024-11-22T22:14:12.938Z,Expansion,Natural Gas,279.0,1200.0,MMscf/d,NaN,NaN,3405,rec_ct0g4dcke70h9fspkeu0,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_ct0g4d...
311,TX,"EOG Resources, Inc.",South Texas Dorado-Verde Pipeline,Construction of 41 miles of new pipeline acros...,Operating,2024-11-25T22:12:43.33Z,New Pipeline,Natural Gas,41.0,1000.0,MMscf/d,NaN,NaN,rec_ct31g27rkv8v7ahcg4l0,rec_ct2fcmr6fvstbp7bv8p0,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_ct2fcm...
312,TX,NaN,Seahawk Pipeline - Thunderdome Laterals,Converting two existing natural gas pipeline l...,Under Construction,2024-11-26T19:36:23.279Z,Conversion,Crude Oil,NaN,486000.0,bpd,NaN,NaN,NaN,rec_ct31os60g3jbpui8294g,NaN,NaN,https://oilandgaswatch.org/pipeline/rec_ct31os...


In [ ]:
import pandas as pd

from dbcp.helpers import get_sql_engine

engine = get_sql_engine()

with engine.connect() as con:
    new_acp_projects = pd.read_sql_table("fossil_infrastructure_projects", con, schema="data_mart")

In [19]:
githash = "a0fc9e0"
old_acp_projects = pd.read_parquet(f"../outputs/{githash}/dev/private_data_warehouse/acp_projects.parquet")

In [24]:
def pct_change_mw_by_status(new, old):

    new_agg = new.groupby("status").capacity_mw.sum()
    old_agg = old.groupby("status").capacity_mw.sum()
    return (new_agg - old_agg) / old_agg * 100

In [25]:
pct_change_mw_by_status(new_acp_projects, old_acp_projects)

status
Advanced Development      -11.403581
Decommissioned              2.578608
Online                      5.934032
Online | Decommissioned     0.416892
Under Construction          5.628867
Name: capacity_mw, dtype: float64

## % capacity change by status and ISO

In [30]:
new_acp_projects.iso_region.value_counts(dropna=False)

None         3588
MISO         1297
PJM          1193
CAISO        1072
ISONE         931
ERCOT         489
NYISO         418
SPP           379
WECC-RMRG       5
Name: iso_region, dtype: int64

In [31]:
for iso_region in new_acp_projects.iso_region.unique():
    new_prj_iso = new_acp_projects[new_acp_projects.iso_region == iso_region]
    old_prj_iso = old_acp_projects[old_acp_projects.iso_region == iso_region]

    pct_change = pct_change_mw_by_status(new_prj_iso, old_prj_iso)

    print(iso_region)
    print(pct_change)
    print()

CAISO
status
Advanced Development      -24.509428
Decommissioned              0.000000
Online                      4.616339
Online | Decommissioned     0.000000
Under Construction        -21.645777
Name: capacity_mw, dtype: float64

None
Series([], Name: capacity_mw, dtype: float64)

PJM
status
Advanced Development      -25.538129
Decommissioned            -22.468304
Online                      4.177580
Online | Decommissioned     0.000000
Under Construction         -3.123384
Name: capacity_mw, dtype: float64

ISONE
status
Advanced Development    0.859603
Decommissioned          0.000000
Online                  3.317459
Under Construction     -6.784373
Name: capacity_mw, dtype: float64

NYISO
status
Advanced Development   -7.567308
Online                  3.476287
Under Construction     -5.726114
Name: capacity_mw, dtype: float64

MISO
status
Advanced Development      -43.586332
Decommissioned              0.000000
Online                      3.340689
Online | Decommissioned     2.5840